# Example of how to use LLMs.

The examples show how to use LLMs from a local model - using Ollama - and from an external model - using OpenAI.

## Using a local model

Using a local model has the advantage of being able to run the model offline. The disadvantage is that the model may not be as good as a model from a large company like OpenAI.

To install Ollama, follow the instructions at their [download homepage](https://ollama.com/download).

Once you have Ollama installed, you need to dpwnload a model. You can see the available models from the [Ollama models page](https://ollama.com/models). For example, to install _llama3_ you will type on your terminal.

```bash
ollama pull mistral
```

You can then use the model directly using ollama's [RESTful API](https://github.com/ollama/ollama/blob/main/docs/api.md).

In this notebook, we will show how to use [LangChain](https://python.langchain.com/v0.2/docs/introduction/) to interact with the model.

In [1]:
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = Ollama(model="llama3", temperature=0.25, keep_alive='10m')
parser = StrOutputParser()

template = ChatPromptTemplate.from_messages([("system", ""), ("user", "Tell me a joke about {topic}")])

chain = template | llm | parser
chain.invoke({"topic": "chickens"})


"Here's one:\n\nWhy did the chicken go to the doctor?\n\nBecause it had a fowl cough!\n\nHope that made you cluck with laughter!"

## Using an external model

Using an external model has the advantage of accesing more powerful models. The disadvantage is that you need to have an internet connection, and pay $$ to use the model.

This notebook assumes that you know how to get an API key from OpenAI, and that the key is stored in an environment variable called `OPENAI_API_KEY`.

Other options are available, like [Groq](https://groq.com/), Mistral and Anthropic.

In [2]:
# read secrets ------------------------------------------------
import json
import os

# you define your on secrets.json file with the following structure
# {
#     "openai": "your-openai-api-key",
#     "groq": "your-groq-api-key"
# }

with open("./secrets.json") as f:
    secrets = json.load(f)

os.environ["OPENAI_API_KEY"] = secrets["openai"]
os.environ["GROQ_API_KEY"] = secrets["groq"]

In [6]:
from langchain_openai import ChatOpenAI

parser = StrOutputParser()

model = ChatOpenAI(model="gpt-4o")


template = ChatPromptTemplate.from_messages([("system", "You communicate in {language}"), ("user", "Tell me a joke about {topic}")])

chain = template | model | parser
chain.invoke({"topic": "chickens", "language": "spanish"})


"Sure! Here's a joke about chickens:\n\nWhy did the chicken go to the seance?\n\nTo talk to the other side!"

## Example: Using a model to summarize a document

In this example, we will use a model to summarize a document. We assume that that the document is local and its called `Personal_Fitness_Tracker.html`.

In [7]:
# Let's first load a document

from langchain_community.document_loaders import UnstructuredFileLoader

files = ["./documents/Personal_Fitness_Tracker.html"]
loader = UnstructuredFileLoader(files, show_progress=True)

docs = loader.load()

In [8]:
# then lets use templates to make an editor that can do different operation on a document

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_community.llms import Ollama

parser = StrOutputParser()

# model = ChatOpenAI(model="gpt-4o")
model = ChatGroq(temperature=0.25, model_name="mixtral-8x7b-32768")
# model = Ollama(model="phi3", temperature=0.25, keep_alive='10m')


template = ChatPromptTemplate.from_messages([("system", "You are a capable copyeditor."), ("user", "Perform the following operation {operation} the following document {document}")])

chain = template | model | parser
chain.invoke({"operation": "summarize in one paragraph", "document": docs[0]})

'The personal fitness tracker market has seen substantial growth in recent years, driven by increasing health awareness and the adoption of wearable technology. Valued at $17.9 billion in 2019, the market is projected to reach $62.1 billion by 2027, growing at a compound annual growth rate of 16.7%. Notable companies in this space include Fitbit, Apple Watch, Garmin, Samsung Gear Fit, and Xiaomi Mi Band. These companies offer various features and cater to different user segments, with Fitbit focusing on a wide range of products and strong brand recognition, Apple Watch on advanced health tracking and seamless integration with Apple devices, Garmin on comprehensive sports and outdoor activity tracking, Samsung Gear Fit on well-rounded features and seamless integration with Samsung devices, and Xiaomi Mi Band on affordability. Despite opportunities in integration with telemedicine, remote patient monitoring, and emerging markets, the market faces challenges such as data privacy concerns,